In [ ]:
!pip install window-slider
!pip install dtaidistance
from dtaidistance import dtw
import time

In [ ]:
import pathlib
import numpy as np
import pandas as pd
import json
import math
import scipy.io
from numpy import linalg as LA
import matplotlib.pyplot as plt
import os
from os.path import join, getsize
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc, roc_auc_score, precision_recall_curve, average_precision_score, cohen_kappa_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from window_slider import Slider
import sys
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
np.set_printoptions(suppress=True)
from sklearn.neighbors import lof
import glob
from sklearn import metrics
import csv

In [ ]:
length_data=dict()

### Seq data

In [ ]:
def ReadDataset_seq(_file_name, _normalize=True):

    df = pd.read_csv(_file_name,header=None)
    abnormal = df.values
######change label position new and old
###################################################################change#########################
#     abnormal_data = abnormal[:,:-1] #change as position of class labels
    abnormal_data = abnormal[:,1:]

#     abnormal_label = abnormal[:,-1]
    abnormal_label = abnormal[:,0]

    print("anomaly",(len(np.where(abnormal_label==-1)[0])))


    # Normal = 1, Abnormal = -1
    
    return abnormal_data,len(abnormal_data),-1*abnormal_label

In [ ]:
if __name__ == '__main__':
    
              for root, dirs, files in os.walk('/dataset/data_warp'):               
                    dirs=['DistalPhalanxOutlineCorrect','MiddlePhalanxOutlineCorrect','ProximalPhalanxOutlineCorrect','Earthquakes','PhalangesOutlinesCorrect','Strawberry','ToeSegmentation1','ToeSegmentation2','DodgerLoopGame','ECG5000','SyntheticControl','ECG200','Wafer','HandOutlines']

                    for dir in dirs:
                        
                        s_precision = []
                        s_recall = []
                        s_f1 = []
                        s_roc_auc = []
                        s_pr_auc = []
                        s_cks = []
                        
                        print("root=",root)
                        print("dir=",dir)
                        
                        for root1,dir1,files in os.walk(os.path.join(root,dir)):
                             print(files)
                             
                             for file in files: 
                                if file.endswith('.csv'):
                                    file_name = os.path.join(root,dir,file)
                                    print(file_name)
                                    abnormal_data,length,abnormal_label = ReadDataset_seq(file_name)
                                    length_data[file.split('.csv')[-2]]=length
                                    print(np.shape(abnormal_data))
                                # print(np.shape(abnormal_label))
                                    file_name1=os.path.join(os.path.join(root,dir),"out_warp",str(os.path.splitext(file)[0])+"_warp.csv")
                                    np.savetxt(file_name1, abnormal_data, delimiter=',',fmt='%10.5f')
#                                     file_name2=os.path.join(os.path.join(root,dir),"out_warp",str(os.path.splitext(file)[0])+"_warp_label.csv")
#                                 # # file_name2=os.path.join(os.path.join(root,dir),"out_warp",str(os.path.splitext(file)[0])+"_warp_revlabel.csv")
#                                     np.savetxt(file_name2, abnormal_label, delimiter=',',fmt='%d')
                             break    
                    break

    

In [ ]:
def ReadDataset_rep(_file_name,rep,root,dir, _normalize=True):

    df = pd.read_csv(_file_name,header=None)
    abnormal_data = df.values

    # Normal = 1, Abnormal = -1
    return abnormal_data


DTW

In [ ]:

def RunModel(w,file,_file_name,root,dir,lenth):
    rep=1

    abnormal_data = ReadDataset_rep(_file_name,rep,root,dir) 
   
    d_nn1=[np.array(0) for i in range(len(abnormal_data))]
    d_nn2=[np.array(0) for i in range(len(abnormal_data))]
    d_nn3=[np.array(0) for i in range(len(abnormal_data))]
    d_nn4=[np.array(0) for i in range(len(abnormal_data))]
    d_nn5=[np.array(0) for i in range(len(abnormal_data))]
    

    start_time = time.time()
    k=0


    for r,t in zip(abnormal_data,range(len(abnormal_data))):
            dist=[] 
            for o in abnormal_data:

                    dist.append(dtw.distance_fast(r,o,window=w))            
            
            dist=np.sort(dist)
            
            d_nn1[k]=dist[1]                     
            d_nn2[k]=dist[5]
            d_nn3[k]=dist[9]
            d_nn4[k]=dist[15]
            d_nn5[k]=dist[20]
      
            
            k+=1                                                ##knn


    print("--- %s seconds ---" % (time.time() - start_time))

  
    return  d_nn1,d_nn2,d_nn3,d_nn4,d_nn5


if __name__ == '__main__':
              
              for root, dirs, files in os.walk('/dataset/data_warp/'):
                    i=0
                    dirs=['DistalPhalanxOutlineCorrect','MiddlePhalanxOutlineCorrect','ProximalPhalanxOutlineCorrect','Earthquakes','PhalangesOutlinesCorrect','Strawberry','ToeSegmentation1','ToeSegmentation2','DodgerLoopGame','ECG5000','SyntheticControl','ECG200']
                  
                    for dir in dirs: 
                        print("root=",root)
                        print("dir=",dir)
                        dir_=os.path.join(root,dir,"out_warp")
                      
                        for root1,dir1,files in os.walk(dir_):
                            print(files)

                            for file in files:
                              i=0
                              best_mean=0
                              if file.endswith("_warp.csv"): 
                                file_name = os.path.join(dir_,file)
                                print("filename",file_name)
                                abnormal_data,length,abnormal_label = ReadDataset_seq(os.path.join(root,dir,str(file.split('_warp.csv')[0]+'.csv')))

                                dist_1, dist_2,dist_3, dist_4, dist_5 = RunModel(int(0.1*len(abnormal_data[0])),file,file_name,root,dir,length_data[file.split('_warp.csv')[-2]])                         
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_1)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_1)
#                                     auc_1=metrics.auc(fpr, tpr)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_2)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_2)
#                                     auc_2=metrics.auc(fpr, tpr)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_3)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_3)
#                                     auc_3=metrics.auc(fpr, tpr)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_4)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_4)
#                                     auc_4=metrics.auc(fpr, tpr)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_5)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_5)
#                                     auc_5=metrics.auc(fpr, tpr)
#                                     mean_auc=np.mean([auc_1,auc_2,auc_3,auc_4,auc_5])
#                                     if best_mean<auc_1:
#                                             best_mean=auc_1
#                                             best_w=w
#                                 print(best_w)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                              
                                  
#                               
#                             i=i+1
                            break
                    i=i+1          
                    break         


LOF

In [ ]:
import time
def RunModel(_file_name,root,dir,lenth):
    rep=0

    abnormal_data = ReadDataset_rep(_file_name,rep,root,dir) 


    d_nn1=[np.array(0) for i in range(len(abnormal_data))]
    d_nn2=[np.array(0) for i in range(len(abnormal_data))]
    d_nn3=[np.array(0) for i in range(len(abnormal_data))]
    d_nn4=[np.array(0) for i in range(len(abnormal_data))]
    d_nn5=[np.array(0) for i in range(len(abnormal_data))]

    k=0
    start_time = time.time()

# ###########lof
    clf = lof.LocalOutlierFactor(n_neighbors=1, n_jobs=2,contamination=0.5)
    y_temp = clf._fit_predict(abnormal_data)
    score=clf.negative_outlier_factor_
    dist=-1*score
    d_nn1=dist 
    clf = lof.LocalOutlierFactor(n_neighbors=3, n_jobs=2,contamination=0.5)
    y_temp = clf._fit_predict(abnormal_data)
    score=clf.negative_outlier_factor_
    dist=-1*score 
    d_nn2=dist
    clf = lof.LocalOutlierFactor(n_neighbors=5, n_jobs=2,contamination=0.5)
    y_temp = clf._fit_predict(abnormal_data)
    score=clf.negative_outlier_factor_
    dist=-1*score
    d_nn3=dist
    clf = lof.LocalOutlierFactor(n_neighbors=7, n_jobs=2,contamination=0.5)
    y_temp = clf._fit_predict(abnormal_data)
    score=clf.negative_outlier_factor_
    dist=-1*score 
    d_nn4=dist
    clf = lof.LocalOutlierFactor(n_neighbors=11, n_jobs=2,contamination=0.5)
    y_temp = clf._fit_predict(abnormal_data)
    score=clf.negative_outlier_factor_
    dist=-1*score
    d_nn5=dist

    print("--- %s seconds ---" % (time.time() - start_time))


    return  d_nn1,d_nn2,d_nn3,d_nn4,d_nn5

    

if __name__ == '__main__':
              
              for root, dirs, files in os.walk('/dataset/data_warp/'):
                   
                    dirs=['DistalPhalanxOutlineCorrect','MiddlePhalanxOutlineCorrect','ProximalPhalanxOutlineCorrect','Earthquakes','PhalangesOutlinesCorrect','Strawberry','ToeSegmentation1','ToeSegmentation2','DodgerLoopGame','ECG5000','SyntheticControl','Wafer','HandOutlines']
                   
                    for dir in dirs:
                        
                        
                        print("root=",root)
                        print("dir=",dir)
                        dir_=os.path.join(root,dir,"out_warp")
                        for root1,dir1,files in os.walk(dir_):
                            print(files)
                            for file in files:
                              
                            
                              if file.endswith("_warp.csv"): 
                                file_name = os.path.join(dir_,file)
                                print("filename",file_name)
                                dist_1,dist_2,dist_3,dist_4,dist_5= RunModel(file_name,root,dir,length_data[file.split('_warp.csv')[-2]])
                                
                                abnormal_data,length,abnormal_label = ReadDataset_seq(os.path.join(root,dir,str(file.split('_warp.csv')[0]+'.csv')))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_1)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_1)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_2)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_2)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_3)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_3)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_4)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_4)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_5)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_5)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                
                             
                            break
                               
                    break         
                                
                                
                                 
                                


ANN

In [ ]:
def _create_autoencoder(time_series, name, seed):
        #encoder
        # conv16@7
        x = Conv1D(filters=16, kernel_size=5, strides=1, padding="same", kernel_initializer=initializers.he_normal(seed=seed), data_format='channels_last')(time_series)
        x = LeakyReLU(alpha=0.3)(x)
        # conv16@5
        x = Conv1D(filters=16, kernel_size=5, strides=1, padding="same", kernel_initializer=initializers.he_normal(seed=seed), data_format="channels_last")(x)
        x = LeakyReLU(alpha=0.3)(x)
        maxpool1_fanin = x._keras_shape[-2]
        # maxpooling@3
        x = MaxPool1D(pool_size=3, padding='same')(x)
        # conv32@3
        x = Conv1D(filters=32, kernel_size=3, strides=1, padding="same", kernel_initializer=initializers.he_normal(seed=seed), data_format="channels_last")(x)
        x = LeakyReLU(alpha=0.3)(x)
        # conv32@3
        x = Conv1D(filters=32, kernel_size=3, strides=1, padding="same", kernel_initializer=initializers.he_normal(seed=seed), data_format="channels_last")(x)
        x = LeakyReLU(alpha=0.3)(x)
        # maxpooling@0.1L
        maxpool2_fanin = x._keras_shape[-2]
        pool_width = int(np.ceil(0.1 * maxpool2_fanin))
        #signet pooling
        x, mask = MaxPoolingWithArgmax1D(pool_size=pool_width, padding="same")(x) #M,16
        num_channels = x._keras_shape[-1]

        x = Flatten()(x) # 16M length
        num_nodes = x._keras_shape[-1]
        # dense@rep_len
        #rep = Dense(units=10, kernel_initializer=initializers.he_normal(seed=seed))(x)
        rep = Dense(units=40, kernel_initializer=initializers.he_normal(seed=seed))(x)

        # decoder
        dense_upsample = Dense(units=num_nodes, kernel_initializer=initializers.he_normal(seed=seed))(rep) #160
        x = Reshape((int(num_nodes/num_channels),num_channels))(dense_upsample) #M,16
        # upsampling@0.1L
        #x = UpSampling1D(size=pool_width)(x)
        x = MaxUnpooling1D(pool_width)([x, mask])
        upsample1_fanout = x._keras_shape[-2]
        # crop excess from right end.
        x = Cropping1D(cropping=(0, upsample1_fanout - maxpool2_fanin))(x) # L,16
        # conv32@3
        x = Conv1D(filters=32, kernel_size=3, strides=1, padding="same", kernel_initializer=initializers.he_normal(seed=seed), data_format="channels_last")(x)
        x = LeakyReLU(alpha=0.3)(x)
        # conv32@3
        x = Conv1D(filters=16, kernel_size=3, strides=1, padding="same", kernel_initializer=initializers.he_normal(seed=seed), data_format="channels_last")(x)
        x = LeakyReLU(alpha=0.3)(x)
        # upsampling@3
        x = UpSampling1D(size=3)(x)
        # Crop excess from the right end.
        upsample2_fanout = x._keras_shape[-2]
        x = Cropping1D(cropping=(0, upsample2_fanout - maxpool1_fanin))(x)
        # conv16@5
        x = Conv1D(filters=16, kernel_size=5, strides=1, padding="same", kernel_initializer=initializers.he_normal(seed=seed), data_format="channels_last")(x)
        x = LeakyReLU(alpha=0.3)(x)
        # conv16@7
        x = Conv1D(filters=1, kernel_size=7, strides=1, padding="same", kernel_initializer=initializers.he_normal(seed=seed), data_format="channels_last")(x)
        out = Lambda(lambda y:y, name=name)(x)

        return rep, out



def RunModel(_abnormal_data, _file_name):
        model=[]
        model_path=os.path.dirname(_file_name)
        series_length=len(_abnormal_data[0])
        time_series_1 = Input(shape=(series_length,1))
        _abnormal_org=_abnormal_data
        _abnormal_org=np.expand_dims(_abnormal_org,axis=-1)
        random.shuffle(_abnormal_data)
        _abnormal_data=np.expand_dims(_abnormal_data,axis=-1)
        rep_layer_1, AE1_out = _create_autoencoder(time_series_1, 'AE1', 1)

        model = Model(inputs=time_series_1, outputs=[AE1_out])

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=100)
        mc = ModelCheckpoint(str(model_path+'best_model.h5'), monitor='val_loss', mode='min', verbose=0, save_best_only=True)
        losses = {
                'AE1':'mean_squared_error',
                }
        model.compile(optimizer=Adam(lr=0.001), loss=losses)
        model.fit(_abnormal_data,_abnormal_data, validation_split=0.2, epochs=2000,verbose=0,callbacks=[es,mc])

        model1=load_model(str(model_path+'best_model.h5'),custom_objects={"MaxPoolingWithArgmax1D": layers.MaxPoolingWithArgmax1D,"MaxUnpooling1D": layers.MaxUnpooling1D})
        inputs = model1.get_layer(index=0).input
        output1 = model1.get_layer(index=12).output
        output2=model1.get_layer(index=26).output
        
        outputs=[output1, output2]
        intermediate_model = Model(inputs=inputs, outputs=outputs)
        print(intermediate_model.summary())
        rep, recons=intermediate_model.predict(_abnormal_org)
        recons=np.squeeze(recons)
        del model
        del intermediate_model
        return rep, recons


if __name__ == '__main__':


    for root, dirs, _ in os.walk('/dataset/data_warp/'):
                dirs=['DistalPhalanxOutlineCorrect','MiddlePhalanxOutlineCorrect','ProximalPhalanxOutlineCorrect','Earthquakes','PhalangesOutlinesCorrect','Strawberry','ToeSegmentation1','ToeSegmentation2','DodgerLoopGame','ECG5000','SyntheticControl','Wafer','HandOutlines']

      
                for dir in dirs:
                    print("root=",root)
                    print("dir=",dir)
                    dir_=os.path.join(root,dir,"out_warp")
                    for root1,dir1,files in os.walk(dir_):

                           for file in files:

                              if file.endswith("_warp.csv"):
                              # if file.endswith(".csv"):
                                file_name = os.path.join(dir_,file)
                                # file_name = os.path.join(os.path.join(root,dir),file)
#                                 print("filename",file_name)

                                final_error = []
                                final_recons=[]
                                abnormal_data = ReadDataset_seq(file_name)


                                embed, recons = RunModel(np.array(abnormal_data),_file_name=file_name)

                                file_name1=os.path.join(os.path.join(root,dir),"ann/",str(os.path.splitext(file)[0])+"_recons.csv")
                                print(file_name1)
                                np.savetxt(file_name1,recons, delimiter=',',fmt='%10.5f')

                                file_name1=os.path.join(os.path.join(root,dir),"ann/",str(os.path.splitext(file)[0])+"_embed.csv")
                                print(file_name1)
                                np.savetxt(file_name1, embed, delimiter=',',fmt='%10.5f')


                            break
                break



In [ ]:
import time
def RunModel(_file_name,root,dir,lenth):
    rep=1
    abnormal_data = ReadDataset_rep(_file_name,rep,root,dir) 

    d_nn1=[np.array(0) for i in range(len(abnormal_data))]
    d_nn2=[np.array(0) for i in range(len(abnormal_data))]
    d_nn3=[np.array(0) for i in range(len(abnormal_data))]
    d_nn4=[np.array(0) for i in range(len(abnormal_data))]
    d_nn5=[np.array(0) for i in range(len(abnormal_data))]

    k=0
    start_time = time.time()



    for r,t in zip(abnormal_data,range(len(abnormal_data))):                    ####knn
            dist=[]

            # dist_arg=[]
            for o in abnormal_data:
                    dist.append(np.linalg.norm(r-o))              
                    
            # dist_arg=(np.argsort(dist))
            dist=np.sort(dist)
            
            d_nn1[k]=dist[1]
            d_nn2[k]=dist[3]
            d_nn3[k]=dist[5]
            d_nn4[k]=dist[7]
            d_nn5[k]=dist[11]


            k+=1                                             ##knn
            
            
    print("--- %s seconds ---" % (time.time() - start_time))   


    return  d_nn1,d_nn2,d_nn3,d_nn4,d_nn5


if __name__ == '__main__':
              
              for root, dirs, files in os.walk('/datasset/data_warp/'):
                     dirs=['DistalPhalanxOutlineCorrect','MiddlePhalanxOutlineCorrect','ProximalPhalanxOutlineCorrect','Earthquakes','PhalangesOutlinesCorrect','Strawberry','ToeSegmentation1','ToeSegmentation2','DodgerLoopGame','ECG5000','SyntheticControl','ECG200']

                     for dir in dirs:
                        
                        
                        print("root=",root)
                        print("dir=",dir)
                        dir_=os.path.join(root,dir,"ann/")
                        for root1,dir1,files in os.walk(dir_):
                            print(files)
                            for file in files:
                              
                              
                              if file.endswith("_warp_embed.csv"): 
                                file_name = os.path.join(dir_,file)
                                print("filename",file_name)
                                dist_1,dist_2,dist_3,dist_4,dist_5 = RunModel(file_name,root,dir,length_data[file.split('_warp_embed.csv')[-2]])
                                
                                abnormal_data,length,abnormal_label = ReadDataset_seq(os.path.join(root,dir,str(file.split('_warp_embed.csv')[0]+'.csv')))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_1)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_1)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_2)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_2)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_3)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_3)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_4)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_4)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_5)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_5)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                            
                            break
                               
                     break         
                                
                              

### sub-seq data

In [ ]:
def ReadDataset_roll(_file_name_annot,anom_size):
    bucket_size=50
    base=os.path.basename(_file_name_annot) 
    base_name=base.split('Annotations_')[1]
    file_name=os.path.join(os.path.dirname(_file_name_annot),base_name)
    label_new=[]
    value=[]
    with open(file_name) as infile:
        value = [float(line.strip('\n')) for line in infile if line]
    abnormal_data=np.array(value)
    print("org len",len(abnormal_data))
    
    with open(_file_name_annot) as infile:
            indice = [int(line.strip('\n')) for line in infile if line]
        
    abnormal_label=np.ones(len(abnormal_data))

    for i in indice:
                abnormal_label[i:i+anom_size]=-1
       
#     bucket_size=anom_size+20  
    abnorm_data=np.empty((0,bucket_size),float)
    abnorm_label=np.empty((0,bucket_size),float)
    overlap_count = 0
    slider = Slider(bucket_size,overlap_count)
    slider1 = Slider(bucket_size,overlap_count)
    slider.fit(abnormal_data) 
    slider1.fit(abnormal_label)
    plt.plot(abnormal_data)
    plt.show()
    plt.plot(abnormal_label)
    plt.show()
    while True:
        window_data1 = slider.slide() 
        window_label1= slider1.slide()
        if  (len(window_data1)<bucket_size) and len(window_data1)>bucket_size/2:
          window_data=np.pad(window_data1,[0,(bucket_size-len(window_data1))])
          window_label=np.pad(window_label1,[0,(bucket_size-len(window_label1))])
          abnorm_data= np.append(abnorm_data,[window_data], axis=0)
          abnorm_label= np.append(abnorm_label,[window_label], axis=0)
          break
        else:
          if len(window_data1)==0 or len(window_data1)<bucket_size/2:
                break
          abnorm_data=np.append(abnorm_data,[window_data1], axis=0)
          abnorm_label=np.append(abnorm_label,[window_label1], axis=0)
            
    label=np.sign(np.sum(abnorm_label,axis=1))
    combined_data=np.column_stack((label,abnorm_data))

    return combined_data, anom_count
  

In [ ]:
if __name__ == '__main__':
              anom_track=dict()
              for root, dirs, files in os.walk('/dataset/discorddataset/'):
                    for dir in dirs:                        
                        print("root=",root)
                        print("dir=",dir)  
                        anom_dict={'Marotta_Valve_Tek_14': 128,'Marotta_Valve_Tek_17': 128, 'chfdbchf15': 200, 'ann_gun_CentroidA_1': 150, 'Patient_respiration2': 150, 'Patient_respiration': 100, 'Marotta_Valve_Tek_16': 128, 'dutch_power_demand': 800}
                        for file in glob.glob(str(root+dir+'/Annotations_*.txt')): 
                                        base=os.path.basename(file)
                                        base=base.split('Annotations_')[1]
                                        base=base.split('.txt')[0]
                                        print(file)
                                        abnormal_data, a_count = ReadDataset_roll(file,anom_dict[base])
                                        anom_track[base]=a_count
                                        print(np.shape(abnormal_data))
                                        np.savetxt(os.path.join(root,dir,str(base+'.csv')), abnormal_data, delimiter=',',fmt='%10.5f')
                    break

In [ ]:

def RunModel(w,file,_file_name,root,dir,lenth):
    rep=1

    
    
    d_nn1=[np.array(0) for i in range(len(abnormal_data))]
    d_nn2=[np.array(0) for i in range(len(abnormal_data))]
    d_nn3=[np.array(0) for i in range(len(abnormal_data))]
    d_nn4=[np.array(0) for i in range(len(abnormal_data))]
    d_nn5=[np.array(0) for i in range(len(abnormal_data))]

    
    start_time = time.time()
    k=0

    for r,t in zip(abnormal_data,range(len(abnormal_data))):
            dist=[]  
            for o in abnormal_data:
                dist.append(dtw.distance(r,o,window=w))  
            

            dist=np.sort(dist)
            
            d_nn1[k]=dist[1]                     
            d_nn2[k]=dist[5]
            d_nn3[k]=dist[9]
            d_nn4[k]=dist[15]
            d_nn5[k]=dist[20]
  
            k+=1                                                ##knn
            


    print("--- %s seconds ---" % (time.time() - start_time))
 
    return  y_pred_arr1,d_nn1,y_pred_arr2,d_nn2,y_pred_arr3,d_nn3,y_pred_arr4,d_nn4,y_pred_arr5,d_nn5

    

if __name__ == '__main__':
              
              for root, dirs, files in os.walk('/dataset/data_warp/'):
                    i=0
                    dirs=['seq_data']
                    for dir in dirs: 
                        print("root=",root)
                        print("dir=",dir)
                        dir_=os.path.join(root,dir,"out_warp")
                      
                        for root1,dir1,files in os.walk(dir_):
                            print(files)
#                             w=[7,13,20,20,16,8,8,9]
                            files=['dutch_power_demand_warp.csv','Marotta_Valve_Tek_14_warp.csv','ann_gun_CentroidA_1_warp.csv',  'chfdbchf15_warp.csv','Patient_respiration2_warp.csv', 'Patient_respiration_warp.csv', 'Marotta_Valve_Tek_16_warp.csv',  'Marotta_Valve_Tek_17_warp.csv']
                            for file in files:
                              i=0
                              best_mean=0
                              if file.endswith("_warp.csv"): 
                                file_name = os.path.join(dir_,file)
                                print("filename",file_name)
                                abnormal_data,length,abnormal_label = ReadDataset_seq(os.path.join(root,dir,str(file.split('_warp.csv')[0]+'.csv')))

                                y_pred1, dist_1,y_pred2, dist_2,y_pred3, dist_3,y_pred4, dist_4,y_pred5, dist_5 = RunModel(int(0.1*len(abnormal_data[0])),file,file_name,root,dir,length_data[file.split('_warp.csv')[-2]])                         
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_1)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_1)
#                                     auc_1=metrics.auc(fpr, tpr)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_2)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_2)
#                                     auc_2=metrics.auc(fpr, tpr)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_3)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_3)
#                                     auc_3=metrics.auc(fpr, tpr)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_4)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_4)
#                                     auc_4=metrics.auc(fpr, tpr)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_5)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_5)
#                                     auc_5=metrics.auc(fpr, tpr)
#                                     mean_auc=np.mean([auc_1,auc_2,auc_3,auc_4,auc_5])
#                                     if best_mean<auc_1:
#                                             best_mean=auc_1
#                                             best_w=w
#                                 print(best_w)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))

                                  
#                       
                            i=i+1
                            break         
                    break         
